In [10]:
import numpy as np
from keras import layers
from keras.layers import  Dense,Flatten, Conv2D
from keras.layers import MaxPooling2D, Dropout
from keras.utils import np_utils,print_summary
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import pandas as pd

import keras.backend as K

In [3]:
    data = pd.read_csv("C:\\Users\\super\\train_foo.csv")
    dataset = np.array(data)
    np.random.shuffle(dataset)
    X = dataset
    Y = dataset
    X = X[:, 1:2501]
    Y = Y[:, 0]

In [4]:
X_train = X[0:12000, :]
X_train = X_train / 255.
X_test = X[12000:13201, :]
X_test = X_test / 255.

In [5]:
# Reshape
Y = Y.reshape(Y.shape[0], 1)
Y_train = Y[0:12000, :]
Y_train = Y_train.T
Y_test = Y[12000:13201, :]
Y_test = Y_test.T

In [6]:
print("number of training examples = " + str(X_train.shape[0]))
print("number of test examples = " + str(X_test.shape[0]))
print("X_train shape: " + str(X_train.shape))
print("Y_train shape: " + str(Y_train.shape))
print("X_test shape: " + str(X_test.shape))
print("Y_test shape: " + str(Y_test.shape))

number of training examples = 12000
number of test examples = 1199
X_train shape: (12000, 2500)
Y_train shape: (1, 12000)
X_test shape: (1199, 2500)
Y_test shape: (1, 1199)


In [7]:
image_x = 50
image_y = 50

train_y =np_utils.to_categorical(Y_train)
test_y =np_utils.to_categorical(Y_test)
train_y =train_y.reshape(train_y.shape[1], train_y.shape[2])
test_y = test_y.reshape(test_y.shape[1], test_y.shape[2])
X_train = X_train.reshape(X_train.shape[0], 50, 50, 1)
X_test = X_test.reshape(X_test.shape[0], 50, 50, 1)
print("X_train shape: " + str(X_train.shape))
print("X_test shape: " + str(X_test.shape))
print("train_y shape:" + str(train_y.shape))

X_train shape: (12000, 50, 50, 1)
X_test shape: (1199, 50, 50, 1)
train_y shape:(12000, 12)


In [8]:
def keras_model(image_x,image_y):
    num_of_classes=12
    #each step followed by next step
    model=Sequential()
    #adding convolutional layer with 32 diff layers with kernel of 5,5
    model.add(Conv2D(32,(5,5),input_shape=(image_x,image_y,1),activation='relu'))
    #max of 2 values and further reduces redimensionality
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same'))
    model.add(Conv2D(64,(5,5),activation='relu'))
    model.add(MaxPooling2D(pool_size=(5,5),strides=(5,5),padding='same'))
    #if they are in matrix form convert into vectors
    model.add(Flatten())
    #relu performs well
    model.add(Dense(1024,activation='relu'))
    #reduces overfitting
    model.add(Dropout(0.6))
    model.add(Dense(num_of_classes,activation='softmax'))#to generalize output in n no of classes
    
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    filepath="HandEmoji.h5"
    #used in very big models
    checkpoint1=ModelCheckpoint(filepath,monitor='val_acc',verbose=1,save_best_only=True,mode='max')
    callbacks_list=[checkpoint1]
    
    return model,callbacks_list

In [9]:
model,callbacks_list=keras_model(image_x,image_y)
model.fit(X_train,train_y,validation_data=(X_test,test_y),epochs=2,batch_size=64,callbacks=callbacks_list)
scores=model.evaluate(X_test,test_y,verbose=0)
print("CNN Error: %.2f%%"%(100-scores[1]*100))
print_summary(model)
model.save('HandEmoji.h5')

W1013 19:07:21.965264 12152 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1013 19:07:21.993863 12152 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1013 19:07:21.996939 12152 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1013 19:07:22.037046 12152 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4267: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W1013 19:07:22.117617 12152 deprecation_wrapper.py:119] From C:\ProgramData\Ana

Train on 12000 samples, validate on 1199 samples
Epoch 1/1
12000/12000 [==============================] - 64s 5ms/step - loss: 0.1607 - acc: 0.9518 - val_loss: 4.3309e-04 - val_acc: 1.0000

Epoch 00001: val_acc improved from -inf to 1.00000, saving model to HandEmoji.h5
CNN Error: 0.00%
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 46, 46, 32)        832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 19, 19, 64)        51264     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (